## 爬取新浪网的新闻

+ 使用 selenium 请求内容
+ 使用 bs4 解析 html

### 大致思路

1. 爬取滚动
1. 判断时间是否在两周内，如果是则爬取

准备

In [4]:
import bs4
import requests_html
import asyncio
import requests



SINA_URL = 'https://news.sina.com.cn/roll/'

# browser.close()

找出链接, 顺便写一个获取最新新闻链接的函数，从第六个链接开始忽略前面的非新闻链接

In [2]:


async def __get_latest_news_urls():
    s = requests_html.AsyncHTMLSession()
    r = await s.get(SINA_URL)
    await r.html.arender()
    bs = bs4.BeautifulSoup(r.html.html, 'html.parser')
    return bs.find_all('a')[6:]

links = await(__get_latest_news_urls())
print(links)


[<a href="https://finance.sina.com.cn/stock/s/2020-04-10/doc-iircuyvh6964012.shtml" target="_blank">新华联商业版图逆势扩张 无奈之下瘦身减持自救</a>, <a href="https://finance.sina.com.cn/stock/jsy/2020-04-10/doc-iircuyvh6963609.shtml" target="_blank">创业板指跌逾2% 家用电器板块涨幅领先</a>, <a href="https://news.sina.com.cn/c/2020-04-10/doc-iirczymi5494335.shtml" target="_blank">109秒被抢空!杭州发7500万元消费券再次刷新申领速度</a>, <a href="https://finance.sina.com.cn/china/gncj/2020-04-10/doc-iirczymi5494039.shtml" target="_blank">国家邮政局：全国96.6%的乡镇已建有快递网点</a>, <a href="https://finance.sina.com.cn/stock/observe/2020-04-10/doc-iirczymi5493293.shtml" target="_blank">陕西黑猫多项财务指标下滑引问询 项目转固1年难投产</a>, <a href="https://finance.sina.com.cn/china/gncj/2020-04-10/doc-iirczymi5493215.shtml" target="_blank">习近平：要加强安全生产监管 分区分类加强安全监管执法</a>, <a href="https://finance.sina.com.cn/roll/2020-04-10/doc-iircuyvh6962784.shtml" target="_blank">穆迪报告：国内外消费者需求下降影响 政策缓解民企难题</a>, <a href="https://finance.sina.com.cn/money/future/fmnews/2020-04-10/doc-iircuyvh6961896.shtml

思考一下数据的维度

1. 标题
1. 类别
2. 时间
4. 关键字
5. 责任编辑
6. 来源
7. 相关专题
8. 正文

总之提取信息相当麻烦，因为格式不是固定的，上述元素也不一定在所有新闻中都出现

这里随便选一个全部都有的新闻当例子

In [5]:
sample_url = "https://finance.sina.com.cn/roll/2020-04-03/doc-iimxxsth3469383.shtml"
r = requests.get(sample_url)
bs = bs4.BeautifulSoup(r.text.encode(r.encoding).decode("utf8","ignore"))

In [6]:
import re
import datetime
import requests

class SinaNews:
    def __init__(self, url: str):
        self.url = url
        r = requests.get(url)
        self.bs = bs4.BeautifulSoup(r.text.encode(r.encoding).decode("utf8","ignore"))
    async def go(self):
#         print(self.bs.prettify())
        
        self.main_title = self.bs.find('title').text
        date_source = self.bs.find('div', 'date-source')
        if date_source is None:
            date_source = self.bs.find('div', 'artInfo')
        date_text = date_source.find('span', 'date')
        if date_text is None:
            date_text = date_source.find('span', id='pub_date')
#         print(date_source, date_text)
#         self.html = self.bs.prettify()

        if date_text is not None:
            date_text = date_text.text.strip()
#             print(date_text)
            date_text = date_text.replace(' ', '')
            
            if '年' in date_text:
                self.publish_date = datetime.datetime.strptime(date_text, '%Y年%m月%d日%H:%M')
            else:
                self.publish_date = datetime.datetime.strptime(date_text, '%Y-%m-%d%H:%M:%S')
        self.source = date_source.find_all()[1].text
        content_list = self.bs.find_all('p')
        self.content = "".join([ tag.text for tag in content_list if not 'class' in tag.attrs ])
        key_words_list = self.bs.find('div', 'article-bottom clearfix')
        if key_words_list is not None:
            key_words_list = key_words_list.find_all('a')
            self.key_words = " ".join([ tag.text for tag in key_words_list if not 'class' in tag.attrs])
        
        editor_source = self.bs.find(string=re.compile('责任编辑'))
        if editor_source is not None:
            editor_source = editor_source.split('责任编辑：')
            self.article_editor = editor_source[-1].strip()
        
        self.category = self.bs.find('div', 'channel-path')
        if self.category is not None:
            self.category = self.category.find('a')
        if self.category is not None:
            self.category = self.category.text
        
        relative_topics_tags = self.bs.find('div', attrs={
            'data-sudaclick': 'content_relativetopics_p'
        })
        if relative_topics_tags is not None:
            relative_topics_tags = relative_topics_tags.find_all('a')
#             print(relative_topics_tags)
            self.relative_topics = " ".join(tag.text for tag in relative_topics_tags)
        self.data = dict()
        for k in self.__dict__:
            if k != 'bs' and k != 'url' and k != 'data':
                self.data[k] = self.__dict__[k]
#                 if k != 'content':
#                     print(k, self.__dict__[k])
        
#         pass
r = SinaNews(sample_url)
await r.go()

写入 csv 文件


In [7]:
import csv
import pandas as pd
import numpy as np

sample_csv_file = "./sample.csv"

def dump(sampleNews: SinaNews, sinaNewsList: list):
    d = dict()
    attrs = sampleNews.data.keys()
    for k in attrs:
        d[k] = []
    for n in sinaNewsList:
        for k in attrs:
            if k in n.data:
                d[k].append(n.data[k])
            else:
                d[k].append(None)
    return pd.DataFrame(d, columns=attrs)


df = dump(r, [r])
print('done')
df.to_csv('sample.csv')


done


尝试读取内容

In [8]:
df2 = pd.read_csv(sample_csv_file, index_col=0)
df2

,main_title,publish_date,source,content,key_words,article_editor,category,relative_topics
0,做空机构曾围绕瑞幸掐架：力挺瑞幸的香椽终于承认浑水是对的|做空机构_新浪财经_新浪网,2020-04-03 13:48:00,澎湃新闻,安装新浪财经客户端第一时间接收最全面的市场资讯→【下载地址】 瑞幸咖啡（Nasdaq：LK...,做空机构 瑞幸咖啡,孟然,美股,瑞幸咖啡伪造交易股价暴跌专题


写一个测试函数，爬取最新的几条新闻

In [9]:
url = "https://news.sina.com.cn/c/2020-04-09/doc-iircuyvh6810407.shtml"
n = SinaNews(url)
await n.go()
print(n.data)

{'main_title': '北京玉渊潭公园门票倒卖情况突出 将实名购票|玉渊潭公园|景山公园|北京_新浪新闻', 'publish_date': datetime.datetime(2020, 4, 9, 16, 39), 'source': '新京报', 'content': '\u3000\u3000原标题：玉渊潭公园门票倒卖情况突出，将实名购票\u3000\u3000景山公园也将实名购票。\u3000\u3000新京报快讯 4月9日，北京召开新冠疫情防控第76场发布会。\u3000\u3000北京市公园管理中心副主任、新闻发言人张亚红表示，市属11家公园在清明假期接待游客数量比去年同期减少70%，游客在游园时也配合戴口罩、测温。这是因为市属公园提前采取网上预约措施。\u3000\u3000倒卖玉渊潭公园门票情况比较突出，已向公安机关报警。相关人员禁止进入所有市属公园。\u3000\u30004月11日，玉渊潭将实行实名购票。景山公园也将实名购票。\u3000\u3000新京报记者 李玉坤更多猛料！欢迎扫描左方二维码关注新浪新闻官方微信（xinlang-xinwen）违法和不良信息举报电话：4000520066\n                    举报邮箱：jubao@vip.sina.comCopyright © 1996-2020 SINA CorporationAll Rights Reserved  新浪公司 版权所有 ', 'key_words': '玉渊潭公园 景山公园 北京', 'article_editor': '郑亚鹏', 'category': ' 社会万象', 'relative_topics': '聚焦新型冠状病毒肺炎疫情'}


In [10]:
import datetime

async def __test_latest_news(limit_pages = 1000):
    def get_page_url(pid):
        pid = int(pid)
        return "https://news.sina.com.cn/roll/#pageid=153&lid=2509&k=&num=50&page={}".format(pid)
    pid = 0
    
    
    async def next_page(pid):
        pid += 1
        s = requests_html.AsyncHTMLSession()
        r = await s.get(get_page_url(pid))
        await r.html.arender()
        bs = bs4.BeautifulSoup(r.html.html, 'html.parser')
        links = bs.find_all('a')[6:]
        return pid, links
    
    
    cnt = 0
    today = datetime.date.today()
    tasks = []
    while True:
        pid, links = await next_page(pid)
        for link in links:
            print(link.text, link['href'])
            n = SinaNews(link['href'])
            task = asyncio.create_task(n.go())
            tasks.append(task)
            cnt += 1
            if cnt % 50 == 0:
                break
            if cnt == limit_pages:
                break
        if cnt == limit_pages:
            break
            
            
    res = []
    for task in tasks:
        n = await task
        res.append(n)
            
    return res



In [ ]:
res = await __test_latest_news(20)

测试基本完成，但是滚动页面最多只能实现50页。分析选择往日回顾的模式，采用另一种方案

In [11]:
def getNewsFromDate(d: datetime.date):
    def date_to_datetime(d):
        return datetime.datetime(d.year, d.month, d.day)

    def timestamp(d):
        return (date_to_datetime(d) - datetime.datetime(1970, 1, 1)) / datetime.timedelta(seconds=1)
    
    dt = date_to_datetime(d)
    
    
    stime = d + datetime.timedelta(hours=4)
    etime = d - datetime.timedelta(days=1)
    ctime = dt
    
    url = '''https://news.sina.com.cn/roll/#pageid=153&lid=2509&etime={}&stime={}&
    ctime={}&date={}&k=&num=50'''.format(timestamp(etime), timestamp(stime), timestamp(ctime), d.strftime('%Y-%m-%d'))
    return url


测试一下最近三天，每天10篇新闻

In [21]:
from itertools import chain

async def getNewsFromDaysDuration(daysl: int, daysr: int, limit_num_per_day=0, output_num=5):
    async def getNewsFromRollPage(url, limit_num=0, output_num=5):
        def get_page_url(pid):
            pid = int(pid)
            return url + "&page={}".format(pid)
        
        pid = 0
        async def next_page(pid):
            pid += 1
            with requests_html.AsyncHTMLSession() as s:
                print(pid, get_page_url(pid))
                r = await s.get(get_page_url(pid))
                
                await r.html.arender()
                bs = bs4.BeautifulSoup(r.html.html, 'html.parser')
                links = bs.find_all('a')[9:]
                
#             print(links)
            return pid, links


        cnt = 0
        today = datetime.date.today()
        tasks = []
        res = []
        while True:
            pid, links = await next_page(pid)
            for link in links:
                if cnt % output_num == 0:
                    print(f"#{cnt}", link.text, link['href'])
                
                try:
                    n = SinaNews(link['href'])
                    res.append(n)
                    tasks.append(asyncio.create_task(n.go()))
                    cnt += 1
                except:
                    print("something wrong with news url: {}".format(link))
                    
                if cnt == limit_num:
                    break
                if cnt % 50 == 0:
                    break
            if cnt == limit_num:
                break
        
        for task in tasks:
            await task
            
        return res
    
    
    today = datetime.date.today()
    res = []
    for i in range(daysl, daysr+1):
        print("day {} from today".format(i))
        d = today - datetime.timedelta(days=i)
        url = getNewsFromDate(d)
        try:
            res += await getNewsFromRollPage(url, limit_num_per_day, output_num)
        except Exception as e:
            print(e)
            print("something wrong when finding news from day {}".format(i))
    
    return res

In [13]:
news = await getNewsFromDaysDuration(3, 10)

day 1 from today
1 https://news.sina.com.cn/roll/#pageid=153&lid=2509&etime=1586304000.0&stime=1586390400.0&
    ctime=1586390400.0&date=2020-04-09&k=&num=50&page=1


/home/weaverzhu/.local/lib/python3.8/site-packages/requests/sessions.py:421: RuntimeWarning: coroutine 'AsyncHTMLSession.close' was never awaited
  self.close()


#0 美移民律师：很多中国客户因疫情失业 也回不了中国 https://news.sina.com.cn/w/2020-04-10/doc-iirczymi5399549.shtml
#5 中国要“接管”全球？美前外交官：担忧被极大夸大 https://finance.sina.com.cn/world/gjcj/2020-04-10/doc-iirczymi5398964.shtml
day 2 from today
1 https://news.sina.com.cn/roll/#pageid=153&lid=2509&etime=1586217600.0&stime=1586304000.0&
    ctime=1586304000.0&date=2020-04-08&k=&num=50&page=1
#0 充电桩：新基建 迈向新能源汽车时代 https://finance.sina.com.cn/stock/stockzmt/2020-04-09/doc-iirczymi5206529.shtml
#5 为遏制疫情蔓延 阿曼关闭首都所有陆路出入口 https://news.sina.com.cn/w/2020-04-09/doc-iirczymi5203902.shtml
day 3 from today
1 https://news.sina.com.cn/roll/#pageid=153&lid=2509&etime=1586131200.0&stime=1586217600.0&
    ctime=1586217600.0&date=2020-04-07&k=&num=50&page=1
#0 任泽平分析全球疫情：欧美陆续现拐点 但有长尾特征 https://finance.sina.com.cn/review/mspl/2020-04-08/doc-iimxxsth4181599.shtml
#5 长城证券：市场驱动因素有望从流动性转向基本面 https://finance.sina.com.cn/stock/stockzmt/2020-04-08/doc-iimxyqwa5620161.shtml


In [ ]:
print(news)

In [14]:
df = dump(r, news)
df.to_csv('sample2.csv')
df.head()


,main_title,publish_date,source,content,key_words,article_editor,category,relative_topics
0,美移民律师：很多中国客户因疫情失业 也回不了中国|疫情|新冠肺炎|美国_新浪新闻,2020-04-10 00:00:00,环球网,原标题：美国移民律师：很多中国客户因为疫情失业，也回不了中国 美国CNN在一篇最新报道...,疫情 新冠肺炎 美国,吴金明,国内新闻,聚焦新型冠状病毒肺炎疫情 全球多国爆发新冠肺炎疫情 美国成疫情“震中”
1,林义相：“启动股市，接力房市”的政策取向不应该变|疫情_新浪财经_新浪网,2020-04-09 23:59:00,新浪财经-自媒体综合,来源：乐趣投资 关注乐趣投资的人，都能找到人生乐趣 “启动股市，接力房市”，是中国经...,疫情 股票市场,常福强,基金,None
2,立陶宛新增43例新冠肺炎确诊病例 累计955例|立陶宛|新冠肺炎_新浪新闻,2020-04-09 23:59:00,央视,原标题：立陶宛新增43例新冠肺炎确诊病例 累计955例 当地时间4月9日，波罗的海新闻...,立陶宛 新冠肺炎,吴金明,国际新闻,聚焦新型冠状病毒肺炎疫情 全球多国爆发新冠肺炎疫情
3,警方通报鲍某某性侵事件：决定再次立案 目前仍在侦查_新浪财经_新浪网,2020-04-09 23:58:00,新浪财经,如何在结构性行情中开展投资布局？新浪财经《基金直播间》，邀请基金经理在线路演解读市场。 新...,,王帅,证券,None
4,武汉“解封”首日到底多少人出城？流向何处？|航班|旅客列车|新冠肺炎_新浪新闻,2020-04-09 23:57:00,红星新闻,原标题：武汉“解封”首日到底多少人出城？流向何处？ 4月8日零时，全国新冠肺炎疫情防控...,航班 旅客列车 新冠肺炎 武汉 湖北,吴金明,国内新闻,聚焦新型冠状病毒肺炎疫情


查看缺失情况

In [15]:
for attr in df:
    print(attr, df[attr].isnull().sum())

main_title 0
publish_date 0
source 0
content 0
key_words 0
article_editor 1
category 0
relative_topics 14


测试收集一天的所有新闻，新浪新闻每天只有50页保存，每页50个新闻，因此一天只有2500新闻最多可以保存

In [16]:
news = await getNewsFromDaysDuration(1, 14, 500, 10)

day 1 from today
1 https://news.sina.com.cn/roll/#pageid=153&lid=2509&etime=1586304000.0&stime=1586390400.0&
    ctime=1586390400.0&date=2020-04-09&k=&num=50&page=1


/home/weaverzhu/.local/lib/python3.8/site-packages/requests/sessions.py:421: RuntimeWarning: coroutine 'AsyncHTMLSession.close' was never awaited
  self.close()


#0 美移民律师：很多中国客户因疫情失业 也回不了中国 https://news.sina.com.cn/w/2020-04-10/doc-iirczymi5399549.shtml
#10 爱奇艺股价攀升；瑞穗认为其收入确认方法与行业一致 https://finance.sina.com.cn/stock/usstock/c/2020-04-09/doc-iircuyvh6868545.shtml
#20 “华为汽车”被曝有望今年落地 华为回应：进行中 https://finance.sina.com.cn/chanjing/gsnews/2020-04-10/doc-iircuyvh6868873.shtml
#30 美媒：这场疫情物资争夺战，穷国被富国挤得靠边站 https://news.sina.com.cn/w/2020-04-09/doc-iirczymi5397094.shtml
#40 股票基金再现一日售罄 认购金额超上限80亿 https://finance.sina.com.cn/roll/2020-04-09/doc-iircuyvh6866605.shtml
2 https://news.sina.com.cn/roll/#pageid=153&lid=2509&etime=1586304000.0&stime=1586390400.0&
    ctime=1586390400.0&date=2020-04-09&k=&num=50&page=2
#50 南非团结基金将为困难家庭提供粮食救济 https://finance.sina.com.cn/roll/2020-04-09/doc-iircuyvh6866616.shtml
#60 武汉“封城”之前，我们究竟做了什么？ https://news.sina.com.cn/c/2020-04-09/doc-iirczymi5395971.shtml
#70 拉卡拉2019净利润同比增34.5% https://finance.sina.com.cn/stock/s/2020-04-09/doc-iircuyvh6864893.shtml
#80 2.3万亿美元 美联储又撒钱了 https://finance.sina.com.cn/stock/stockptd/2020-04-09/doc

In [17]:
len(news)

500

In [20]:
df = dump(r, news)
df.head()
# df.describe()
df.to_csv('0410.csv')

In [ ]:
for i in range(2, 14):
    news = await getNewsFromDaysDuration(i, i+1, 2500, 20)
    df = dump(r, news)
    df.to_csv(f'{i}.csv')

day 2 from today
1 https://news.sina.com.cn/roll/#pageid=153&lid=2509&etime=1586217600.0&stime=1586304000.0&
    ctime=1586304000.0&date=2020-04-08&k=&num=50&page=1


/home/weaverzhu/.local/lib/python3.8/site-packages/requests/sessions.py:421: RuntimeWarning: coroutine 'AsyncHTMLSession.close' was never awaited
  self.close()


#0 充电桩：新基建 迈向新能源汽车时代 https://finance.sina.com.cn/stock/stockzmt/2020-04-09/doc-iirczymi5206529.shtml
#20 荣盛发展：为何不对一笔6.25亿款项计提坏账？ https://finance.sina.com.cn/stock/s/2020-04-08/doc-iircuyvh6673505.shtml
#40 爱奇艺CEO龚宇回应被做空：邪不压正 看最后谁赢！ https://finance.sina.com.cn/stock/relnews/us/2020-04-08/doc-iirczymi5202401.shtml
2 https://news.sina.com.cn/roll/#pageid=153&lid=2509&etime=1586217600.0&stime=1586304000.0&
    ctime=1586304000.0&date=2020-04-08&k=&num=50&page=2
#60 美国确诊近40万例！美专家预计今秋形势会明显改善 https://news.sina.com.cn/w/2020-04-08/doc-iircuyvh6672251.shtml
#80 意大利因新冠肺炎疫情禁止难民船靠岸 https://news.sina.com.cn/w/2020-04-08/doc-iircuyvh6670599.shtml
3 https://news.sina.com.cn/roll/#pageid=153&lid=2509&etime=1586217600.0&stime=1586304000.0&
    ctime=1586304000.0&date=2020-04-08&k=&num=50&page=3
#100 OPEC+会议召开在即：特朗普推脱减产 俄沙再生分歧 https://finance.sina.com.cn/wm/2020-04-08/doc-iirczymi5198495.shtml
#120 尼日利亚向国际金融机构寻求35亿美元预算资金 https://finance.sina.com.cn/roll/2020-04-09/doc-iirczymi5197578.shtml
#140 武磊：已康复但未